In [ ]:
# Copyright (c) 2020 ZZH

In [ ]:
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, AveragePooling2D, MaxPool2D
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model
import os
import numpy as np
import math

In [ ]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
epochs = 100
lr = 0.1
batch_size = 128
REGULARIZER  = 0.0001
checkpoint_save_path =  './Model/DenseNet121/'
log_dir = os.path.join("Model","DenseNet121_logs")

In [ ]:
#数据导入及数据增强
cifar10 = tf.keras.datasets.cifar10
(x_train,y_train),(x_test,y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)
mean = [125.307, 122.95, 113.865]  #np.mean()
std = [62.9932, 62.0887, 66.7048]  #np.std()
for i in range(3):
    x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
    x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]

DataGenTrain = tf.keras.preprocessing.image.ImageDataGenerator(
               rotation_range = 15,
               width_shift_range = 0.125,
               height_shift_range = 0.125,
               horizontal_flip = True,
               vertical_flip = False,
               shear_range=0.125,
               zoom_range = 0.125)
DataGenTrain.fit(x_train)

In [ ]:
def scheduler(epoch):  #HTD(-6,3)衰减
    start = -6.0
    end = 3.0
    return lr / 2.0 * (1- math.tanh( (end-start)*epoch/epochs + start))

In [ ]:
class DenseLayer(Model):
    def __init__(self,channels):
        super(DenseLayer,self).__init__()
        self.b1 = BatchNormalization(momentum=0.9)
        self.a1 = Activation('relu')
        self.c1 = Conv2D(filters=4*channels, kernel_size=1, strides=1, padding='same',
                         kernel_initializer="he_normal",kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER))
        self.b2 = BatchNormalization(momentum=0.9)  
        self.a2 = Activation('relu')
        self.c2 = Conv2D(filters=channels, kernel_size=3, strides=1, padding='same', 
                         kernel_initializer="he_normal",kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER))
    def call(self,inputs):
        x = self.b1(inputs)
        x = self.a1(x)
        x = self.c1(x)
        x = self.b2(x)
        x = self.a2(x)
        outputs = self.c2(x)
        return outputs

In [ ]:
class DenseBlock(Model):
    def __init__(self,layerNum,growth_rate):
        super(DenseBlock,self).__init__()
        self.models = []
        self.layerNum = layerNum
        for _ in range(self.layerNum):
            self.models.append(DenseLayer(channels=growth_rate))
    def call(self,inputs):
        outputs = inputs
        for i in range(self.layerNum):
            x = self.models[i](outputs)
            outputs = tf.concat([outputs,x], axis = -1)
        return outputs

In [ ]:
class TransitionLayer(Model):
    def __init__(self,channels,compression):
        super(TransitionLayer,self).__init__()
        self.b1 = BatchNormalization(momentum=0.9)
        self.a1 = Activation('relu')
        self.c1 = Conv2D(filters=int(channels*compression), kernel_size=1, strides=1, padding='same',
                         kernel_initializer="he_normal",kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER))
        self.p1 = AveragePooling2D(pool_size=2, strides=2)
    def call(self,inputs):
        x = self.b1(inputs)
        x = self.a1(x)
        x = self.c1(x)
        outputs = self.p1(x)
        return outputs

In [ ]:
class DenseNet(Model):
    def __init__(self,layerNum,growth_rate):
        super(DenseNet,self).__init__()
        self.growth_rate = growth_rate
        self.layerNum = layerNum
        self.c1 = Conv2D(filters=2*self.growth_rate, kernel_size=3, strides=1, padding='same', 
                        kernel_initializer="he_normal",kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER))
        self.b1 = BatchNormalization(momentum=0.9)
        self.a1 = Activation('relu')
        
        self.channels = 2 * self.growth_rate
        self.blocks = Sequential()
        self.blocks.add(DenseBlock(layerNum = self.layerNum[0], growth_rate = self.growth_rate))
        self.channels += self.growth_rate * self.layerNum[0]
        self.blocks.add(TransitionLayer(channels = self.channels, compression=0.5))
        self.blocks.add(DenseBlock(layerNum = self.layerNum[1], growth_rate = self.growth_rate))
        self.channels += self.growth_rate * self.layerNum[1]
        self.blocks.add(TransitionLayer(channels = self.channels, compression=0.5))
        self.blocks.add(DenseBlock(layerNum = self.layerNum[2], growth_rate = self.growth_rate))
        self.channels += self.growth_rate * self.layerNum[2]
        self.blocks.add(TransitionLayer(channels = self.channels, compression=0.5))
        self.blocks.add(DenseBlock(layerNum = self.layerNum[3], growth_rate = self.growth_rate))
        
        self.b2 = BatchNormalization(momentum=0.9)
        self.a2 = Activation('relu')
        self.p1 = GlobalAveragePooling2D()
        self.f1 = Dense(10,activation='softmax',kernel_initializer="he_normal",
                        kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER))
    def call(self,inputs):
        x = self.c1(inputs)
        x = self.b1(x)
        x = self.a1(x)
        x = self.blocks(x)
        x = self.b2(x)
        x = self.a2(x)
        x = self.p1(x)
        y = self.f1(x)
        return y

In [ ]:
model = DenseNet(layerNum = [6,12,24,16], growth_rate = 32)

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9, nesterov=True, clipnorm=2.),
              loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
              metrics=['accuracy'])

callbacks = [
            tf.keras.callbacks.LearningRateScheduler(scheduler),  #学习率衰减表
            #tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, min_lr=0.0001, patience=10, cooldown=0)
            tf.keras.callbacks.ModelCheckpoint(     #模型保存
                filepath = checkpoint_save_path,
                save_weights_only = False,
                monitor = 'val_accuracy',
                save_best_only = True),
#             tf.keras.callbacks.EarlyStopping(       #早停
#                 monitor = 'val_accuracy',
#                 patience=15, 
#                 baseline=None),
            tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_images=False)  #保存计算图
]

hist = model.fit(DataGenTrain.flow(x_train,y_train,batch_size=batch_size,shuffle=True),
                 epochs=epochs,
                 validation_data=(x_test,y_test),
                 validation_freq=1,
                 callbacks=callbacks)

model.summary()

In [ ]:
#结果可视化
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use({'figure.figsize':(6,4)})

plt.plot(hist.history['loss'], label='loss')
plt.plot(hist.history['val_loss'], label='val_loss')
plt.legend()
plt.show()
plt.plot(hist.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()

In [ ]:
#tensorboard可视化
#!tensorboard --logdir=./Model/DenseNet121_logs
#http://localhost:6006/

In [ ]:
print('best result: {:.2f}%  ({}epochs)'.format(100*max(hist.history['val_accuracy']),1+hist.history['val_accuracy'].index(max(hist.history['val_accuracy']))))
# best result: 94.91%  (100epochs)  [6,12,24,16]  121-32